In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [12]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = 'NKQCJOO2KRZKDUZLH0XCPJ3SC00TX3ZHWEABS3R0Y0UUOZMO' # your Foursquare ID
CLIENT_SECRET = 'YOXAM3QCM35J0QHIFJTEEHEJXQPEASMO1AF4YF2HXPIGGWK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [4]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", "Los Angeles, CA", "Seattle, WA",]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") # coffee PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [9]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [14]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Cafes in {city} = ", results[city]['response']['totalResults'])

Total number of Cafes in New York, NY =  226
Total number of Cafes in Los Angeles, CA =  201
Total number of Cafes in Seattle, WA =  205


In [15]:
maps[cities[0]]

In [16]:
maps[cities[1]]

In [17]:
maps[cities[2]]

From the inital analysis we can see that coffee places density is higher in New York and Seattle compared to LA as LA is more scattered across. 